In [104]:
import pandas as pd
import numpy as np
from datetime import timedelta,datetime
import warnings
warnings.filterwarnings("ignore")

In [105]:
schedule=pd.read_excel("328-H New_ Summary_Schedule_3min_Headway 40 buses 20240815.xlsx",sheet_name="Schedule")
schedule

,bus_id,trip_no,start_stop,end_stop,route_name,direction,route_id,distance,start_day,start_time,...,Spread,Depot,Zone,Service,Brand,Sch. kms,Dead Kms,Total Kms,Shift Category,schedule
0,1,1,Attibele Bus Stand,Hosakote Bus Stand (Old),328-H,UP,328-HUP,40,1,04:00:00,...,01:30:00,DEPOT,ZONE,SUBURBAN,DIESEL,40,0,40,Night Out Shift,328-H/1
1,1,2,Hosakote Bus Stand (Old),Attibele Bus Stand,328-H,DN,328-HDN,40,1,05:36:00,...,03:06:00,DEPOT,ZONE,SUBURBAN,DIESEL,80,0,80,Night Out Shift,328-H/1
2,1,3,Attibele Bus Stand,Hosakote Bus Stand (Old),328-H,UP,328-HUP,40,1,07:12:00,...,05:02:00,DEPOT,ZONE,SUBURBAN,DIESEL,120,0,120,Night Out Shift,328-H/1
3,1,4,Hosakote Bus Stand (Old),Attibele Bus Stand,328-H,DN,328-HDN,40,1,09:33:00,...,07:23:00,DEPOT,ZONE,SUBURBAN,DIESEL,160,0,160,Night Out Shift,328-H/1
4,1,5,Attibele Bus Stand,Depot-32,328-H,NaN,328-H,7,1,11:30:00,...,07:35:00,DEPOT,ZONE,SUBURBAN,DIESEL,160,7,167,Night Out Shift,328-H/1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,40,4,Attibele Bus Stand,Hosakote Bus Stand (Old),328-H,UP,328-HUP,40,1,12:57:00,...,06:02:00,DEPOT,ZONE,SUBURBAN,DIESEL,120,7,127,General Shift,328-H/40
368,40,5,Hosakote Bus Stand (Old),Attibele Bus Stand,328-H,DN,328-HDN,40,1,14:42:00,...,07:47:00,DEPOT,ZONE,SUBURBAN,DIESEL,160,7,167,General Shift,328-H/40
369,40,6,Attibele Bus Stand,Kadugudi,328-H,UP,328-HUP,30,1,16:45:00,...,09:30:00,DEPOT,ZONE,SUBURBAN,DIESEL,190,7,197,General Shift,328-H/40
370,40,7,Kadugudi,Attibele Bus Stand,328-H,DN,328-HDN,30,1,18:10:00,...,10:55:00,DEPOT,ZONE,SUBURBAN,DIESEL,220,7,227,General Shift,328-H/40


In [106]:
schedule.columns

Index(['bus_id', 'trip_no', 'start_stop', 'end_stop', 'route_name',
       'direction', 'route_id', 'distance', 'start_day', 'start_time',
       'run_time', 'end_day', 'end_time', 'headway', 'min_break_time',
       'ready_day', 'ready_time', 'actual_break_time', 'Shift', 'is_dead_trip',
       'Deadtrip', 'Steering', 'Spread', 'Depot', 'Zone', 'Service', 'Brand',
       'Sch. kms', 'Dead Kms', 'Total Kms', 'Shift Category', 'schedule'],
      dtype='object')

In [107]:
limit_df=pd.read_excel("headway_limit.xlsx")
limit_df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,30,30,30,30,30,30,30,15,30,30,...,30,30,30,30,30,30,30,30,30,30


In [108]:
def convert_to_timedelta(time_str):
    return pd.to_timedelta(time_str)
schedule[schedule["headway"].apply(convert_to_timedelta)>timedelta(minutes=15)]["bus_id"]
sc=[]

In [109]:
def select_nearest_stop(input):
    if input == 'Attibele':
        return 'Kadugudi'
    elif input == 'Hosakote':
        return 'Sarjapura'

def format_time(time):
    if isinstance(time, timedelta):
        total_seconds = int(time.total_seconds())
        hours, remainder = divmod(total_seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        return f'{hours:02}:{minutes:02}:{seconds:02}'
    elif isinstance(time, datetime):
        return time.strftime('%H:%M:%S')

def get_runtime():
    return timedelta(minutes=60)

def get_distance():
    return 30

# def get_available(new_start_time,start_stop_to_check):
#     # Group by 'start_stop'
#     grouped = schedule.groupby('start_stop')['start_time'].apply(list)

#     while(True):
#         # Check if new_start_time is in the list for the specific start_stop
#         if start_stop_to_check in grouped:
#             if new_start_time in grouped[start_stop_to_check]:
#                 new_start_time=pd.to_datetime(, format='%H:%M:%S')

In [110]:
def get_travel_time(current_time):
    current_minutes = current_time.total_seconds() // 60
    if 4 * 60 <= current_minutes < 6 * 60:
        return timedelta(minutes=90)
    elif 6 * 60 <= current_minutes < 7 * 60:
        return timedelta(minutes=100)
    elif 7 * 60 <= current_minutes < 10 * 60:
        return timedelta(minutes=110)
    elif 10 * 60 <= current_minutes < 16 * 60:
        return timedelta(minutes=100)
    elif 16 * 60 <= current_minutes < 20 * 60:
        return timedelta(minutes=110)
    elif 20 * 60 <= current_minutes < 22 * 60:
        return timedelta(minutes=100)
    else:
        return timedelta(minutes=100)

k=1
for hour in range(24):
    limit = limit_df[hour].iloc[0]  # Assuming the limit is in the first row for each hour
    # print(limit)
    # Identify rows where headway exceeds the limit
    exceeded_df = schedule[(pd.to_datetime(schedule['start_time']).apply(lambda x: x.hour) == hour) & 
                              (pd.to_datetime(schedule['headway']).dt.minute > limit)]
    for index, row in exceeded_df.iterrows():
        # Calculate the new start time by adding half the headway to the previous start time
        k=k+1
        bus_id=f'ABD{k}'
        shift=row['Shift']
        trip=row['trip_no']
        start_stop=row['start_stop']
        new_start_time = (pd.to_datetime(row['start_time'], format='%H:%M:%S') + 
                          pd.Timedelta(minutes=row['headway']/2)).time()
        
        if(shift=="General"):
            print(exceeded_df)
            print("General")
            trip_no=0
            for i in range(trip,1,-1):
                if(start_stop=="Hosakote Bus Stand (Old)"):
                    start_stop="Attibele Bus Stand"
                elif(start_stop=="Attibele Bus Stand"):
                    start_stop="Hosakote Bus Stand (Old)"
                if(trip%2!=0):
                    min_break=30
                else:
                    min_break=5
                run_time=get_runtime(new_start_time)
                start_time=new_start_time-run_time-min_break

            if(start_stop=="Hosakote Bus Stand (Old)"):
                trip_no+=1
                run_time=timedelta(minute=5)
                min_break=timedelta(minutes=5)
                start_time=start_time-run_time-min_break
                end_time=start_time+run_time
                ready_time=end_time+min_break
                start_stop="Depot-39"
                end_stop=start_stop
                direction="UP"
                distance=2
                route_id="328-H"
                route_name="N/A"

                sc.append({
                                'bus_id': bus_id,
                                'trip_number': trip_no,
                                'start_stop': start_stop,
                                'end_stop': end_stop,
                                'route_name': route_name,
                                'direction': direction,
                                'route_id': route_id,
                                'distance': distance,
                                'start_time': format_time(start_time),
                                'end_time': format_time(end_time),
                                'runtime': format_time(run_time),
                                'minBreak_time': format_time(min_break),
                                'ready_time': format_time(ready_time),
                                'actualBreak_time': format_time(min_break),
                                'shift_id': 'NightOut 1',
                                'isDeadTrip': False
                                })

            elif(start_stop=="Attibele Bus Stand"):
                trip_no+=1
                run_time=timedelta(minute=5)
                min_break=timedelta(minutes=5)
                start_time=start_time-run_time-min_break
                end_time=start_time+run_time
                ready_time=end_time+min_break
                start_stop="Depot-32"
                end_stop=start_stop
                direction="DN"
                distance=2
                route_id="328-H"
                route_name="N/A"

                sc.append({
                                'bus_id': bus_id,
                                'trip_number': trip_no,
                                'start_stop': start_stop,
                                'end_stop': end_stop,
                                'route_name': route_name,
                                'direction': direction,
                                'route_id': route_id,
                                'distance': distance,
                                'start_time': format_time(start_time),
                                'end_time': format_time(end_time),
                                'runtime': format_time(run_time),
                                'minBreak_time': format_time(min_break),
                                'ready_time': format_time(ready_time),
                                'actualBreak_time': format_time(min_break),
                                'shift_id': 'NightOut 1',
                                'isDeadTrip': False
                                })

            # actual_start=ready_time
            # start_stop=end_stop
            # start_hour = 18
            # start_minute = 0
            # end_hour = 19
            # end_minute = 30
            # start_time_of_day = timedelta(hours=start_hour,minutes=end_minute)              #input
            # end_time_of_day = timedelta(hours=end_hour, minutes=end_minute)
            # s=0
            # while start_time_of_day <= n + get_travel_time(n) <= end_time_of_day:

            #                 for k in range(0,2):
            #                     trip_no+=1
            #                     if(k==0):
            #                         start_stop = 'Attelebe'
            #                         end_stop = 'Hosakote'
            #                         direction='UP'
            #                         route_id = '328-UP'
            #                         starting='Hosakote'


            #                     else:
            #                         start_stop = 'Hosakote'
            #                         end_stop = 'Attebele'
            #                         direction='DN'
            #                         route_id = '328-DN'
            #                         starting='Attebele'

            #                     route_name = '328H'
            #                     runtime = get_travel_time(actual_start)
            #                     min_break = timedelta(minutes=5)
            #                     start_time = actual_start
            #                     distance=40

                                

            #                     if s%2!=0:
            #                         min_break = timedelta(minutes=30)

            #                     end_time = actual_start + runtime
            #                     ready_time = end_time+min_break 


            #                     schedule.append({
            #                 'bus_id': bus_id,
            #                 'trip_number': trip_no,
            #                 'start_stop': start_stop,
            #                 'end_stop': end_stop,
            #                 'route_name': route_name,
            #                 'direction': direction,
            #                 'route_id': route_id,
            #                 'distance': distance,
            #                 'start_time': format_time(start_time),
            #                 'end_time': format_time(end_time),
            #                 'runtime': format_time(runtime),
            #                 'minBreak_time': format_time(min_break),
            #                 'ready_time': format_time(ready_time),
            #                 'actualBreak_time': format_time(min_break),
            #                 'shift_id': 'NightOut 1',
            #                 'isDeadTrip': False
            #             })
            #                     actual_start = ready_time

            #                 n=actual_start+get_travel_time(actual_start) 
                        

            # if(n+2*timedelta(hours=1)-get_travel_time(actual_start)<end_time_of_day):
            #             start_stop=end_stop
            #             end_stop = select_nearest_stop('Attibele')
            #             runtime = get_runtime()
            #             distance = get_distance()
            #             route_name = '328H'
            #             min_break = timedelta(minutes=5)
            #             start_time = actual_start
            #             # # Calculate the duration since the start time
            #             # elapsed_time = actual_start - temp_time

            #             # if elapsed_time > max_runtime_duration:
            #             #     min_break = timedelta(minutes=30)
            #             #     elapsed_time = timedelta()
            #             end_time = actual_start + runtime
            #             ready_time = end_time+min_break

            #             schedule.append({
            #                     'bus_id': bus_id,
            #                     'trip_number': trip_no,
            #                     'start_stop': start_stop,
            #                     'end_stop': end_stop,
            #                     'route_name': route_name,
            #                     'direction': direction,
            #                     'route_id': route_id,
            #                     'distance': distance,
            #                     'start_time': format_time(start_time),
            #                     'end_time': format_time(end_time),
            #                     'runtime': format_time(runtime),
            #                     'minBreak_time': format_time(min_break),
            #                     'ready_time': format_time(ready_time),
            #                     'actualBreak_time': format_time(min_break),
            #                     'shift_id': 'NightOut 1',
            #                     'isDeadTrip': False
            #                     })
            #             actual_start = ready_time

            #             start_stop=end_stop
            #             end_stop = "Attebele"
            #             runtime = get_runtime()
            #             distance = get_distance()
            #             route_name = '328H'
            #             min_break = timedelta(minutes=5)
            #             start_time = actual_start

            #             # if elapsed_time > max_runtime_duration:
            #             #     min_break = timedelta(minutes=30)
                           
            #             end_time = actual_start + runtime
            #             ready_time = end_time+min_break

            #             schedule.append({
            #                     'bus_id': bus_id,
            #                     'trip_number': trip_no,
            #                     'start_stop': start_stop,
            #                     'end_stop': end_stop,
            #                     'route_name': route_name,
            #                     'direction': direction,
            #                     'route_id': route_id,
            #                     'distance': distance,
            #                     'start_time': format_time(start_time),
            #                     'end_time': format_time(end_time),
            #                     'runtime': format_time(runtime),
            #                     'minBreak_time': format_time(min_break),
            #                     'ready_time': format_time(ready_time),
            #                     'actualBreak_time': format_time(min_break),
            #                     'shift_id': 'NightOut 1',
            #                     'isDeadTrip': False
            #                     })
            #             actual_start = ready_time

            # start_stop=end_stop
            # if(start_stop=="Attebele"):
            #             end_stop="Depot 32"
            #             distance=7
            # else:
            #             end_stop="Depot 39"
            #             distance=2
            # trip_no+=1
            # route_id="328H"
            # start_time=ready_time
            # min_break=timedelta(hours=0,minutes=0)
            # runtime=timedelta(minutes=5)
            # end_time=start_time+runtime
            # ready_time=end_time+min_break+timedelta(hours=1,minutes=30)
            # schedule.append({
            #                 'bus_id': bus_id,
            #                 'trip_number': trip_no,
            #                 'start_stop': start_stop,
            #                 'end_stop': end_stop,
            #                 'route_name': route_name,
            #                 'direction': "",
            #                 'route_id': route_id,
            #                 'distance': distance,
            #                 'start_time': format_time(start_time),
            #                 'end_time': format_time(end_time),
            #                 'runtime': format_time(runtime),
            #                 'minBreak_time': format_time(min_break),
            #                 'ready_time': format_time(ready_time),
            #                 'actualBreak_time': format_time(timedelta(hours=1,minutes=30)),
            #                 'shift_id': 'NightOut 1',
            #                 'isDeadTrip': True
            #             })
            # actual_start = ready_time

df = pd.DataFrame(sc)

# print(df)

Empty DataFrame
Columns: [bus_id, trip_no, start_stop, end_stop, route_name, direction, route_id, distance, start_day, start_time, run_time, end_day, end_time, headway, min_break_time, ready_day, ready_time, actual_break_time, Shift, is_dead_trip, Deadtrip, Steering, Spread, Depot, Zone, Service, Brand, Sch. kms, Dead Kms, Total Kms, Shift Category, schedule]
Index: []

[0 rows x 32 columns]
Empty DataFrame
Columns: [bus_id, trip_no, start_stop, end_stop, route_name, direction, route_id, distance, start_day, start_time, run_time, end_day, end_time, headway, min_break_time, ready_day, ready_time, actual_break_time, Shift, is_dead_trip, Deadtrip, Steering, Spread, Depot, Zone, Service, Brand, Sch. kms, Dead Kms, Total Kms, Shift Category, schedule]
Index: []

[0 rows x 32 columns]
Empty DataFrame
Columns: [bus_id, trip_no, start_stop, end_stop, route_name, direction, route_id, distance, start_day, start_time, run_time, end_day, end_time, headway, min_break_time, ready_day, ready_time, ac

In [92]:
hour=2
limit_df[hour].iloc[0]

np.int64(30)